In [1]:
import os
import torch
import random
import datetime
import pandas as pd
import numpy as np

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, get_svd_encoder
from src.embeddings import RecsysEmbedding

experiment_name = 'SessionwiseGRU'
device = 'cuda:2'
seed = 123
pkl_path = '../pkl/'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
torch.__version__

'1.12.1'

# Модель

In [3]:
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)

class SessionwiseGRU(torch.nn.Module):
    def __init__(self, embedding, output_dim=1, dropout = 0.1):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.embedding = embedding
        self.rnn_layer = torch.nn.GRU(
            input_size = embedding.embedding_dim, 
            hidden_size = embedding.embedding_dim, 
            batch_first = True,
            dropout=dropout
        )
        self.out_layer = torch.nn.Linear(embedding.embedding_dim, output_dim)


    def forward(self, batch):
        shp = batch['slates_item_indexes'].shape
        item_embs, user_embs = self.embedding(batch)
        item_embs = item_embs.flatten(-3, -2)
        
        # while training, let out model see the future 
        # while testing, it can see only the 
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2).unsqueeze(0)

#         print(indices.shape, user_embs.shape, item_embs.shape, )
        rnn_out, _ = self.rnn_layer(
            item_embs,
            user_embs,
        )
        return self.out_layer(rnn_out).reshape(shp)

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SessionwiseGRU(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='neural').to('cpu'),
    output_dim=1
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 3610.59it/s]
/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.4000000059604645, 'roc-auc': 0.3333333134651184, 'accuracy': 0.25}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | treshold: 0.51
Test: accuracy: 0.5 | f1: 0.5 | auc: 0.3333333134651184 | 
Val update: epoch: 3 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.54
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SessionwiseGRU(
   (embedding): RecsysEmbedding(
     (item_embeddings): Embedding(5, 32)
   )
   (rnn_layer): GRU(32, 32, batch_first=True, dropout=0.1)
   (out_layer): Linear(in_features=32, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    
    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings
        ),
        output_dim=1
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating SessionwiseGRU with svd embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.17910169064998627, 'roc-auc': 0.5107409954071045, 'accuracy': 0.10133212804794312}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.10012787580490112 | f1: 0.18202951550483704 | auc: 0.555400550365448 | treshold: 0.01
Test: accuracy: 0.09851612895727158 | f1: 0.1793621927499771 | auc: 0.5454712510108948 | 
Val update: epoch: 1 |accuracy: 0.10012787580490112 | f1: 0.18202951550483704 | auc: 0.5753637552261353 | treshold: 0.060000000000000005
Test: accuracy: 0.09851612895727158 | f1: 0.1793621927499771 | auc: 0.5669745206832886 | 
Val update: epoch: 2 |accuracy: 0.8400392532348633 | f1: 0.23419703543186188 | auc: 0.5863630771636963 | treshold: 0.06999999999999999
Test: accuracy: 0.8422603607177734 | f1: 0.22795115411281586 | auc: 0.5774627327919006 | 
Val update: epoch: 3 |accuracy: 0.822583019733429 | f1: 0.2441403716802597 | auc: 0.614797055721283 | treshold: 0.13
Test: accuracy: 0.8254081606864929 | f1: 0.24323298037052155 | auc: 0.6134625673294067 | 
Val update: epoch: 4 |accuracy: 0.7833288908004761 | f1: 0.24473929405212402 | auc: 0.6311878561973572 | treshold: 0.13
Test: accur

/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.15120254456996918, 'roc-auc': 0.4822978973388672, 'accuracy': 0.583422064781189}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.10012787580490112 | f1: 0.18202951550483704 | auc: 0.6200113296508789 | treshold: 0.01
Test: accuracy: 0.09851612895727158 | f1: 0.1793621927499771 | auc: 0.6115891933441162 | 
Val update: epoch: 1 |accuracy: 0.671290934085846 | f1: 0.2415342926979065 | auc: 0.6553618907928467 | treshold: 0.09
Test: accuracy: 0.6666034460067749 | f1: 0.23644991219043732 | auc: 0.6487568020820618 | 
Val update: epoch: 2 |accuracy: 0.790584921836853 | f1: 0.26980507373809814 | auc: 0.6845998764038086 | treshold: 0.15000000000000002
Test: accuracy: 0.7944322228431702 | f1: 0.27365055680274963 | auc: 0.6808114051818848 | 
Val update: epoch: 3 |accuracy: 0.8634876608848572 | f1: 0.25774112343788147 | auc: 0.6984960436820984 | treshold: 0.13
Test: accuracy: 0.8657951354980469 | f1: 0.2498776763677597 | auc: 0.6953215003013611 | 
Val update: epoch: 4 |accuracy: 0.6519463658332825 | f1: 0.2686828374862671 | auc: 0.7068164944648743 | treshold: 0.14
Test: accuracy: 0.65297573804

In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['explicit', 'neural', 'svd']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SessionwiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
        output_dim=1
    ).to(device)


    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating SessionwiseGRU with explicit embeddings


/opt/conda/envs/myenv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Test before learning: {'f1': 0.7154248356819153, 'roc-auc': 0.4887523353099823, 'accuracy': 0.584040641784668}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7557684183120728 | f1: 0.8053059577941895 | auc: 0.8257359266281128 | treshold: 0.46
Test: accuracy: 0.7563776969909668 | f1: 0.807188093662262 | auc: 0.8237989544868469 | 
Val update: epoch: 1 |accuracy: 0.7912736535072327 | f1: 0.839400053024292 | auc: 0.8547525405883789 | treshold: 0.32
Test: accuracy: 0.7886108160018921 | f1: 0.8380451202392578 | auc: 0.8492683172225952 | 
Val update: epoch: 3 |accuracy: 0.8001983165740967 | f1: 0.8592602610588074 | auc: 0.8781238198280334 | treshold: 0.37
Test: accuracy: 0.7990086078643799 | f1: 0.859046995639801 | auc: 0.8736324310302734 | 
Val update: epoch: 5 |accuracy: 0.8118802309036255 | f1: 0.854142427444458 | auc: 0.8925448060035706 | treshold: 0.39
Test: accuracy: 0.8080522418022156 | f1: 0.8516926765441895 | auc: 0.8877182006835938 | 
Val update: epoch: 6 |accuracy: 0.822401225566864 | f1: 0.8596441149711609 | auc: 0.9006321430206299 | treshold: 0.38
Test: accuracy: 0.8147987127304077 | f1: 0.85410594940

...

Val update: epoch: 0 |accuracy: 0.7555265426635742 | f1: 0.8329477310180664 | auc: 0.8019580841064453 | treshold: 0.41000000000000003
Test: accuracy: 0.7569096684455872 | f1: 0.8347606062889099 | auc: 0.7968358993530273 | 
Val update: epoch: 1 |accuracy: 0.775431752204895 | f1: 0.8462468385696411 | auc: 0.8512120246887207 | treshold: 0.33
Test: accuracy: 0.7755773067474365 | f1: 0.8470122814178467 | auc: 0.8489636182785034 | 
Val update: epoch: 2 |accuracy: 0.792095959186554 | f1: 0.8337137699127197 | auc: 0.874822735786438 | treshold: 0.35000000000000003
Test: accuracy: 0.7839196920394897 | f1: 0.8280943632125854 | auc: 0.8681647777557373 | 
Val update: epoch: 4 |accuracy: 0.822957456111908 | f1: 0.8607994318008423 | auc: 0.9023512601852417 | treshold: 0.34
Test: accuracy: 0.8164913654327393 | f1: 0.8561245203018188 | auc: 0.8961309194564819 | 
Val update: epoch: 5 |accuracy: 0.8322691321372986 | f1: 0.8730806708335876 | auc: 0.9116961359977722 | treshold: 0.38
Test: accuracy: 0.82887

/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028



Evaluating SessionwiseGRU with svd embeddings
Test before learning: {'f1': 0.479218453168869, 'roc-auc': 0.5180301666259766, 'accuracy': 0.49341070652008057}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.6453344821929932 | f1: 0.7844417095184326 | auc: 0.6615904569625854 | treshold: 0.01
Test: accuracy: 0.650852382183075 | f1: 0.7885046601295471 | auc: 0.6532622575759888 | 
Val update: epoch: 1 |accuracy: 0.6453344821929932 | f1: 0.7844417095184326 | auc: 0.7502915859222412 | treshold: 0.22
Test: accuracy: 0.650852382183075 | f1: 0.7885046601295471 | auc: 0.7426897287368774 | 
Val update: epoch: 2 |accuracy: 0.6459391713142395 | f1: 0.7845273017883301 | auc: 0.7784627676010132 | treshold: 0.29000000000000004
Test: accuracy: 0.6514085531234741 | f1: 0.788621723651886 | auc: 0.7729441523551941 | 
Val update: epoch: 3 |accuracy: 0.7024863362312317 | f1: 0.8060238361358643 | auc: 0.7925496101379395 | treshold: 0.42000000000000004
Test: accuracy: 0.7035908699035645 | f1: 0.8075788021087646 | auc: 0.7875521183013916 | 
Val update: epoch: 5 |accuracy: 0.731896698474884 | f1: 0.8174015879631042 | auc: 0.8075422048568726 | treshold: 0.41000000000000003
Test: ac

In [10]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items